In [15]:
# encoding: utf-8

import unicodecsv as csv
import json
import sys
import io
import re

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("ISO-8859-1")
# sys.setdefaultencoding("cp1252")
sys.stdout = stdout

In [16]:
csv_reader = csv.reader(open('data/dbpedia-discussion-archive.csv', 'r'))
file_writer = open('data/dbpedia-discussion-archive-questions.txt', 'w')
# csv_writer = csv.writer(open('data/dbpedia-discussion-archive-questions.csv', 'wb'))

question_topics = set()
questions = {}
messages = []
questions_list = []
ignored_subjects = set()

def contains_question_word(word):
    question_words = ['what', 'when', 'why', 'which', 'who', 'how', 'whose', 'whom']
    return any(s in word.lower() for s in question_words)

def contains_ignored_phrases(text):
    return re.search(r'call for|cfp|ann\:|important dates|Dbpedia-discussion Digest', text, re.I)

def is_ignored_subject(subject, ignored_subjects, message):
    if subject in ignored_subjects or subject == '' or subject == '(no subject)':
        return True
    # Removing call for, CFP and Announcements
    elif contains_ignored_phrases(subject) or contains_ignored_phrases(message):
        return True
    return False

def process_message(message):
    # Moved to MBox
#     message = re.sub(r'<[^>]*>', ' ', message, re.MULTILINE) # Remove Tags
#     message = re.sub(r'http[\S\r\n\t]+', ' ', message, re.MULTILINE) # Remove Links or URI etc    
#     message = re.sub(r'\s{2,}', ' ', message, re.MULTILINE) # Remove extra whitespaces
    return 'u' + message.strip()

def is_question_topic(subject, message):
    global question_topics, questions, ignored_subjects

    if is_ignored_subject(subject, ignored_subjects, message):
        ignored_subjects.add(subject)
        return False
    elif subject in question_topics or contains_question_word(subject) or contains_question_word(message):
        if subject not in questions.keys():
            questions[subject] = {
                'thread': []
            }
        questions[subject]['thread'].append(process_message(message))
        return True    
    else:
        ignored_subjects.add(subject)
        return False

In [17]:
count = 0
for line in csv_reader:
    if is_question_topic(line[3], line[4]):
        question_topics.add(line[3])
        count += 1
        messages.append(line[4])
        
print count
print len(questions.keys())

# for i in range(0, 100):
#     print messages[i]

4889
1392


In [18]:
# Writing all subjects separately to a file
for subject in questions.keys():
    # Remove threads with single message and blank subject not doing this for now
#     if len(questions[subject]['thread']) == 1:
#         print subject
#         del questions[subject]
#         continue
    file_writer.write("%s\n" % subject)
#     csv_writer.writerow([subject, ' '.join(questions[subject]['thread'])])
    questions_list.append({'subject': subject, 'thread': questions[subject]['thread'], 'thread_length': len(questions[subject]['thread'])})

with open('data/dbpedia-discussion-archive-questions.json', 'w') as fp:
    json.dump(questions_list, fp)

In [81]:
if re.search(r'call for|cfp|ann\:', 'kaljsdkajd CALL FOR :', re.IGNORECASE):
    print True

True
